In [1]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pandas as pd

# Initialize SparkSession
spark = SparkSession.builder.appName("KMeans Clustering").getOrCreate()

# Get the list of stock ticker symbols in KOSPI
from pykrx import stock
ticker_list = stock.get_index_ticker_list("KOSPI")

# Create an empty DataFrame to store the fundamental data
fundamental_data = []

# Iterate over the ticker symbols and retrieve the fundamental data
for ticker in ticker_list:
    data = stock.get_market_fundamental_by_date("20230523", ticker)
    fundamental_data.append(data)

# Create a Pandas DataFrame from the collected fundamental data
kospi_df = pd.concat(fundamental_data)

# Create a Spark DataFrame from the Pandas DataFrame
kospi_spark_df = spark.createDataFrame(kospi_df)

# Select the required columns for clustering
selected_cols = ["종목코드", "DIV", "BPS", "PER", "EPS", "PBR"]
selected_data = kospi_spark_df.select(selected_cols)

# Perform Standard Scaling
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scaled_data = scaler.fit(selected_data).transform(selected_data)

# Perform K-means Clustering
kmeans = KMeans(featuresCol="scaledFeatures", k=3)
model = kmeans.fit(scaled_data)

# Add the predicted cluster labels to the DataFrame
clustered_data = model.transform(scaled_data)

# Count the number of stocks in each cluster
cluster_count = clustered_data.groupBy("prediction").count()

# Display the number of stocks in each cluster
cluster_count.show()


23/05/30 14:02:07 WARN Utils: Your hostname, yangsejin-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.45 instead (on interface en0)
23/05/30 14:02:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/30 14:02:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/30 14:02:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


TypeError: get_market_fundamental_by_date() missing 1 required positional argument: 'ticker'

23/05/30 14:02:25 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
